In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split


In [ ]:
data_dir = '/content/plates/license-plates/'


In [ ]:
image_files = os.listdir(data_dir)
images = []
labels = []

for img_file in image_files:
    img_path = os.path.join(data_dir, img_file)
    label = os.path.splitext(img_file)[0]  # Dosya uzantısını kaldırarak etiketi elde edin
    images.append(img_path)
    labels.append(label)


In [ ]:
characters = set()
max_label_length = 0

for label in labels:
    characters.update(list(label))
    if len(label) > max_label_length:
        max_label_length = len(label)

characters = sorted(list(characters))
num_chars = len(characters)

print("Benzersiz karakter sayısı:", num_chars)
print("Karakterler:", characters)
print("Maksimum etiket uzunluğu:", max_label_length)


Benzersiz karakter sayısı: 34
Karakterler: ['-', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'V', 'Y', 'Z']
Maksimum etiket uzunluğu: 9


In [ ]:
char_to_num = {char: idx for idx, char in enumerate(characters)}
num_to_char = {idx: char for idx, char in enumerate(characters)}


In [ ]:
label_sequences = []

for label in labels:
    label_seq = [char_to_num[char] for char in label]
    # Maksimum etiket uzunluğuna yastıklama
    label_seq += [-1] * (max_label_length - len(label_seq))
    label_sequences.append(label_seq)

label_sequences = np.array(label_sequences)


In [ ]:
train_images, val_images, train_labels, val_labels, train_label_sequences, val_label_sequences = train_test_split(
    images, labels, label_sequences, test_size=0.1, random_state=42)

print("Eğitim örnek sayısı:", len(train_images))
print("Doğrulama örnek sayısı:", len(val_images))


Eğitim örnek sayısı: 90000
Doğrulama örnek sayısı: 10000


In [ ]:
img_width = 128
img_height = 32
batch_size = 64
downsample_factor = 4  # CNN mimarisine bağlıdır


In [ ]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self, images, labels, label_sequences, batch_size, img_width, img_height, max_label_length):
        self.images = images
        self.labels = labels
        self.label_sequences = label_sequences
        self.batch_size = batch_size
        self.img_width = img_width
        self.img_height = img_height
        self.max_label_length = max_label_length

    def __len__(self):
        return int(np.ceil(len(self.images) / self.batch_size))

    def __getitem__(self, idx):
        batch_x = self.images[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.label_sequences[idx * self.batch_size:(idx + 1) * self.batch_size]

        X_data = np.zeros((len(batch_x), self.img_width, self.img_height, 1), dtype=np.float32)
        labels = np.ones([len(batch_x), self.max_label_length]) * -1
        input_length = np.ones((len(batch_x), 1)) * (self.img_width // downsample_factor - 2)
        label_length = np.zeros((len(batch_x), 1))

        for i, img_path in enumerate(batch_x):
            # Görüntüyü yükleyin ve ön işleyin
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (self.img_width, self.img_height))
            img = img.astype(np.float32) / 255.0
            img = np.expand_dims(img, axis=-1)
            X_data[i] = img

            # Etiket bilgilerini güncelleyin
            labels[i, :] = batch_y[i]
            label_length[i] = len(self.labels[idx * self.batch_size + i])

        inputs = {
            'input_data': X_data,
            'labels': labels,
            'input_length': input_length,
            'label_length': label_length
        }
        outputs = {'ctc': np.zeros([len(batch_x)])}  # CTC kaybı için dummy çıkış

        return inputs, outputs


In [ ]:
from tensorflow.keras import backend as K

# Giriş katmanı
input_data = layers.Input(name='input_data', shape=(img_width, img_height, 1), dtype='float32')

# CNN katmanları
inner = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(input_data)
inner = layers.MaxPooling2D(pool_size=(2, 2))(inner)
inner = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(inner)
inner = layers.MaxPooling2D(pool_size=(2, 2))(inner)

# RNN için şekil değiştirme
conv_shape = inner.get_shape()
inner = layers.Reshape(target_shape=(conv_shape[1], conv_shape[2] * conv_shape[3]))(inner)
inner = layers.Dense(64, activation='relu')(inner)

# RNN katmanları
gru_1 = layers.GRU(128, return_sequences=True, kernel_initializer='he_normal', name='gru1')(inner)
gru_1b = layers.GRU(128, return_sequences=True, go_backwards=True, kernel_initializer='he_normal', name='gru1_b')(inner)
gru1_merged = layers.add([gru_1, gru_1b])

gru_2 = layers.GRU(128, return_sequences=True, kernel_initializer='he_normal', name='gru2')(gru1_merged)
gru_2b = layers.GRU(128, return_sequences=True, go_backwards=True, kernel_initializer='he_normal', name='gru2_b')(gru1_merged)
gru2_merged = layers.concatenate([gru_2, gru_2b])

# Çıkış katmanı
inner = layers.Dense(num_chars, kernel_initializer='he_normal', activation='softmax')(gru2_merged)

# CTC kaybı için girişler
labels = layers.Input(name='labels', shape=[max_label_length], dtype='float32')
input_length = layers.Input(name='input_length', shape=[1], dtype='int64')
label_length = layers.Input(name='label_length', shape=[1], dtype='int64')

# CTC kaybı fonksiyonu
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    return keras.backend.ctc_batch_cost(labels, y_pred, input_length, label_length)

# Kayıp katmanı
loss_out = layers.Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([inner, labels, input_length, label_length])

# Modelin tanımlanması
model = keras.models.Model(inputs=[input_data, labels, input_length, label_length], outputs=loss_out)


AttributeError: 'KerasTensor' object has no attribute 'get_shape'

In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from tensorflow.keras import backend as K

# 2. Veri Yolunun Tanımlanması
data_dir = '/content/plates/license-plates/'

# 3. Görüntü Dosyalarının ve Etiketlerin Yüklenmesi
image_files = os.listdir(data_dir)
images = []
labels = []

for img_file in image_files:
    img_path = os.path.join(data_dir, img_file)
    label = os.path.splitext(img_file)[0]  # Dosya uzantısını kaldırarak etiketi elde edin
    images.append(img_path)
    labels.append(label)

# 4. Benzersiz Karakterlerin ve Maksimum Etiket Uzunluğunun Belirlenmesi
characters = set()
max_label_length = 0

for label in labels:
    characters.update(list(label))
    if len(label) > max_label_length:
        max_label_length = len(label)

characters = sorted(list(characters))
num_chars = len(characters)

print("Benzersiz karakter sayısı:", num_chars)
print("Karakterler:", characters)
print("Maksimum etiket uzunluğu:", max_label_length)

# 5. Karakter-Etkileşim Haritalarının Oluşturulması
char_to_num = {char: idx for idx, char in enumerate(characters)}
num_to_char = {idx: char for idx, char in enumerate(characters)}

# 6. Etiketlerin Sayısal Değerlere Dönüştürülmesi ve Yastıklama
label_sequences = []

for label in labels:
    label_seq = [char_to_num[char] for char in label]
    # Maksimum etiket uzunluğuna yastıklama (-1 ile)
    label_seq += [-1] * (max_label_length - len(label_seq))
    label_sequences.append(label_seq)

label_sequences = np.array(label_sequences, dtype='int64')  # Veri tipini int64 olarak ayarlayın

# 7. Verilerin Eğitim ve Doğrulama Setlerine Bölünmesi
train_images, val_images, train_labels, val_labels, train_label_sequences, val_label_sequences = train_test_split(
    images, labels, label_sequences, test_size=0.1, random_state=42)

print("Eğitim örnek sayısı:", len(train_images))
print("Doğrulama örnek sayısı:", len(val_images))

# 8. Veri Parametrelerinin Belirlenmesi
img_width = 128
img_height = 32
batch_size = 64
downsample_factor = 4  # CNN mimarisine bağlıdır

# 9. Veri Üreticisinin Tanımlanması
class DataGenerator(keras.utils.Sequence):
    def __init__(self, images, labels, label_sequences, batch_size, img_width, img_height, max_label_length):
        self.images = images
        self.labels = labels
        self.label_sequences = label_sequences
        self.batch_size = batch_size
        self.img_width = img_width
        self.img_height = img_height
        self.max_label_length = max_label_length

    def __len__(self):
        return int(np.ceil(len(self.images) / self.batch_size))

    def __getitem__(self, idx):
        batch_x = self.images[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.label_sequences[idx * self.batch_size:(idx + 1) * self.batch_size]

        X_data = np.zeros((len(batch_x), self.img_height, self.img_width, 1), dtype=np.float32)
        the_labels = np.ones([len(batch_x), self.max_label_length], dtype='int64') * -1  # Veri tipini int64 yapın
        input_length = np.ones((len(batch_x), 1), dtype='int64') * np.int64(self.img_width // downsample_factor - 2)
        label_length = np.zeros((len(batch_x), 1), dtype='int64')

        for i, img_path in enumerate(batch_x):
            # Görüntüyü yükleyin ve ön işleyin
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                print(f"Uyarı: {img_path} yüklenemedi.")
                continue
            img = cv2.resize(img, (self.img_width, self.img_height))
            img = img.astype(np.float32) / 255.0
            img = np.expand_dims(img, axis=-1)
            X_data[i] = img

            # Etiket bilgilerini güncelleyin
            the_labels[i, :] = batch_y[i]
            # Gerçek etiket uzunluğunu hesaplayın (padding dışındaki karakter sayısı)
            label_length[i] = np.int64(np.sum(batch_y[i] != -1))

        inputs = {
            'input_data': X_data,
            'the_labels': the_labels,        # Yeni isimle atayın
            'input_length': input_length,
            'label_length': label_length
        }
        outputs = {'ctc': np.zeros([len(batch_x)])}  # CTC kaybı için dummy çıkış

        return inputs, outputs

# 10. Modelin Oluşturulması
# Giriş katmanlarını tanımlayın
input_data = layers.Input(name='input_data', shape=(img_height, img_width, 1), dtype='float32')
the_labels = layers.Input(name='the_labels', shape=[max_label_length], dtype='int64')  # Yeniden adlandırıldı
input_length = layers.Input(name='input_length', shape=[1], dtype='int64')
label_length = layers.Input(name='label_length', shape=[1], dtype='int64')

# CNN katmanları
inner = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(input_data)
inner = layers.MaxPooling2D(pool_size=(2, 2))(inner)
inner = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(inner)
inner = layers.MaxPooling2D(pool_size=(2, 2))(inner)

# RNN için şekil değiştirme
conv_shape = K.int_shape(inner)
inner = layers.Reshape(target_shape=(conv_shape[1], conv_shape[2] * conv_shape[3]))(inner)
inner = layers.Dense(64, activation='relu')(inner)

# RNN katmanları
gru_1 = layers.GRU(128, return_sequences=True, kernel_initializer='he_normal', name='gru1')(inner)
gru_1b = layers.GRU(128, return_sequences=True, go_backwards=True, kernel_initializer='he_normal', name='gru1_b')(inner)
gru1_merged = layers.add([gru_1, gru_1b])

gru_2 = layers.GRU(128, return_sequences=True, kernel_initializer='he_normal', name='gru2')(gru1_merged)
gru_2b = layers.GRU(128, return_sequences=True, go_backwards=True, kernel_initializer='he_normal', name='gru2_b')(gru1_merged)
gru2_merged = layers.concatenate([gru_2, gru_2b])

# Çıkış katmanı
# CTC için blank label eklemek üzere num_chars + 1
inner = layers.Dense(num_chars + 1, kernel_initializer='he_normal', activation='softmax')(gru2_merged)

# CTC kaybı fonksiyonu
def ctc_lambda_func(args):
    labels, y_pred, input_length, label_length = args
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

# Kayıp katmanı
loss_out = layers.Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([the_labels, inner, input_length, label_length])

# Modelin tanımlanması
model = keras.models.Model(inputs=[input_data, the_labels, input_length, label_length], outputs=loss_out)

# 11. Modelin Derlenmesi
model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer='adam')

# Modelin özetini yazdırın
model.summary()

# 12. Veri Üreticilerinin Oluşturulması
train_generator = DataGenerator(train_images, train_labels, train_label_sequences, batch_size, img_width, img_height, max_label_length)
val_generator = DataGenerator(val_images, val_labels, val_label_sequences, batch_size, img_width, img_height, max_label_length)

# 13. Modelin Eğitilmesi
model.fit(train_generator,
          validation_data=val_generator,
          epochs=10)

# 14. Tahmin Modelinin Oluşturulması
# Tahmin modeli, CTC kaybını içermeyen bir modeldir
prediction_model = keras.models.Model(inputs=input_data, outputs=inner)

# 15. Tahminlerin Çözümlenmesi
def decode_batch_predictions(pred):
    input_len = np.ones(pred.shape[0]) * pred.shape[1]
    results = K.ctc_decode(pred, input_length=input_len, greedy=True)[0][0]
    output_text = []

    for res in results:
        # CTC decode sonucu, -1 olanları içermez
        res = res.numpy()
        decoded = ''.join([num_to_char[int(r)] for r in res if r != -1])
        output_text.append(decoded)

    return output_text

# 16. Modelin Test Edilmesi
for batch in val_generator:
    batch_images = batch[0]['input_data']
    preds = prediction_model.predict(batch_images)
    pred_texts = decode_batch_predictions(preds)

    for i in range(len(pred_texts)):
        print("Gerçek Etiket:", val_labels[i])
        print("Tahmin Edilen Etiket:", pred_texts[i])
    break


Benzersiz karakter sayısı: 34
Karakterler: ['-', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'V', 'Y', 'Z']
Maksimum etiket uzunluğu: 9
Eğitim örnek sayısı: 90000
Doğrulama örnek sayısı: 10000


Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_data (InputLayer)   │ (None, 32, 128, 1)     │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_20 (Conv2D)        │ (None, 32, 128, 64)    │            640 │ input_data[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_20          │ (None, 16, 64, 64)     │              0 │ conv2d_20[0][0]        │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_21 (Conv2D)        │ (None, 16, 64, 128)    │         73,856 │ max_pooling2d_20[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_21          │ (None, 8, 32, 128)     │              0 │ conv2d_21[0][0]        │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape_7 (Reshape)       │ (None, 8, 4096)        │              0 │ max_pooling2d_21[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_14 (Dense)          │ (None, 8, 64)          │        262,208 │ reshape_7[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ gru1 (GRU)                │ (None, 8, 128)         │         74,496 │ dense_14[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ gru1_b (GRU)              │ (None, 8, 128)         │         74,496 │ dense_14[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_7 (Add)               │ (None, 8, 128)         │              0 │ gru1[0][0],            │
│                           │                        │                │ gru1_b[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ gru2 (GRU)                │ (None, 8, 128)         │         99,072 │ add_7[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ gru2_b (GRU)              │ (None, 8, 128)         │         99,072 │ add_7[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_7             │ (None, 8, 256)         │              0 │ gru2[0][0],            │
│ (Concatenate)             │                        │                │ gru2_b[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ the_labels (InputLayer)   │ (None, 9)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_15 (Dense)          │ (None, 8, 35)          │          8,995 │ concatenate_7[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_length (InputLayer) │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ label_length (InputLayer) │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ ctc (Lambda)         

 Total params: 692,835 (2.64 MB)

 Trainable params: 692,835 (2.64 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10


ValueError: Exception encountered when calling Lambda.call().

[1mCan not squeeze dim[1], expected a dimension of 1, got 9 for '{{node functional_7_1/ctc_1/Squeeze}} = Squeeze[T=DT_INT64, squeeze_dims=[-1]](data_1)' with input shapes: [?,9].[0m

Arguments received by Lambda.call():
  • inputs=['tf.Tensor(shape=(None, 1), dtype=int64)', 'tf.Tensor(shape=(None, 8, 35), dtype=float32)', 'tf.Tensor(shape=(None, 1), dtype=int64)', 'tf.Tensor(shape=(None, 9), dtype=int64)']
  • mask=['None', 'None', 'None', 'None']
  • training=True

In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from tensorflow.keras import backend as K

# 1. Veri Yolunun Tanımlanması
data_dir = '/content/plates/license-plates/'

# 2. Görüntü Dosyalarının ve Etiketlerin Yüklenmesi
image_files = os.listdir(data_dir)
images = []
labels = []

for img_file in image_files:
    img_path = os.path.join(data_dir, img_file)
    label = os.path.splitext(img_file)[0]  # Dosya uzantısını kaldırarak etiketi elde edin
    images.append(img_path)
    labels.append(label)

# 3. Benzersiz Karakterlerin ve Maksimum Etiket Uzunluğunun Belirlenmesi
characters = set()
max_label_length = 0

for label in labels:
    characters.update(list(label))
    if len(label) > max_label_length:
        max_label_length = len(label)

characters = sorted(list(characters))
num_chars = len(characters)

print("Benzersiz karakter sayısı:", num_chars)
print("Karakterler:", characters)
print("Maksimum etiket uzunluğu:", max_label_length)

# 4. Karakter-Etkileşim Haritalarının Oluşturulması
char_to_num = {char: idx for idx, char in enumerate(characters)}
num_to_char = {idx: char for idx, char in enumerate(characters)}

# 5. Etiketlerin Sayısal Değerlere Dönüştürülmesi ve Yastıklama
label_sequences = []

for label in labels:
    label_seq = [char_to_num[char] for char in label]
    label_seq += [-1] * (max_label_length - len(label_seq))  # Yastıklama
    label_sequences.append(label_seq)

label_sequences = np.array(label_sequences, dtype='int64')  # Veri tipini int64 olarak ayarlayın

# 6. Verilerin Eğitim ve Doğrulama Setlerine Bölünmesi
train_images, val_images, train_labels, val_labels, train_label_sequences, val_label_sequences = train_test_split(
    images, labels, label_sequences, test_size=0.1, random_state=42)

print("Eğitim örnek sayısı:", len(train_images))
print("Doğrulama örnek sayısı:", len(val_images))

# 7. Veri Parametrelerinin Belirlenmesi
img_width = 128
img_height = 32
batch_size = 64
downsample_factor = 4  # CNN mimarisine bağlıdır

# 8. Veri Üreticisinin Tanımlanması
class DataGenerator(keras.utils.Sequence):
    def __init__(self, images, labels, label_sequences, batch_size, img_width, img_height, max_label_length):
        self.images = images
        self.labels = labels
        self.label_sequences = label_sequences
        self.batch_size = batch_size
        self.img_width = img_width
        self.img_height = img_height
        self.max_label_length = max_label_length

    def __len__(self):
        return int(np.ceil(len(self.images) / self.batch_size))

    def __getitem__(self, idx):
        batch_x = self.images[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.label_sequences[idx * self.batch_size:(idx + 1) * self.batch_size]

        X_data = np.zeros((len(batch_x), self.img_height, self.img_width, 1), dtype=np.float32)
        the_labels = np.ones([len(batch_x), self.max_label_length], dtype='int64') * -1
        input_length = np.ones((len(batch_x), 1), dtype='int64') * (self.img_width // downsample_factor - 2)
        label_length = np.zeros((len(batch_x), 1), dtype='int64')

        for i, img_path in enumerate(batch_x):
            # Görüntüyü yükleyin ve ön işleyin
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                print(f"Uyarı: {img_path} yüklenemedi.")
                continue
            img = cv2.resize(img, (self.img_width, self.img_height))
            img = img.astype(np.float32) / 255.0
            img = np.expand_dims(img, axis=-1)
            X_data[i] = img

            # Etiket bilgilerini güncelleyin
            the_labels[i, :] = batch_y[i]
            label_length[i] = np.int64(np.sum(batch_y[i] != -1))

        inputs = {
            'input_data': X_data,
            'label_length': label_length,  # Düzeltildi
            'input_length': input_length,
            'the_labels': the_labels       # Düzeltildi
        }
        outputs = {'ctc': np.zeros([len(batch_x)])}  # CTC kaybı için dummy çıkış

        return inputs, outputs

# 9. Modelin Oluşturulması
input_data = layers.Input(name='input_data', shape=(img_height, img_width, 1), dtype='float32')
the_labels = layers.Input(name='the_labels', shape=(max_label_length,), dtype='int64')
input_length = layers.Input(name='input_length', shape=(1,), dtype='int64')
label_length = layers.Input(name='label_length', shape=(1,), dtype='int64')

# CNN katmanları
inner = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(input_data)
inner = layers.MaxPooling2D(pool_size=(2, 2))(inner)
inner = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(inner)
inner = layers.MaxPooling2D(pool_size=(2, 2))(inner)

# RNN için şekil değiştirme
conv_shape = K.int_shape(inner)
inner = layers.Reshape(target_shape=(conv_shape[2], conv_shape[1] * conv_shape[3]))(inner)
inner = layers.Dense(64, activation='relu')(inner)

# RNN katmanları
gru_1 = layers.GRU(128, return_sequences=True, kernel_initializer='he_normal', name='gru1')(inner)
gru_1b = layers.GRU(128, return_sequences=True, go_backwards=True, kernel_initializer='he_normal', name='gru1_b')(inner)
gru1_merged = layers.add([gru_1, gru_1b])

gru_2 = layers.GRU(128, return_sequences=True, kernel_initializer='he_normal', name='gru2')(gru1_merged)
gru_2b = layers.GRU(128, return_sequences=True, go_backwards=True, kernel_initializer='he_normal', name='gru2_b')(gru1_merged)
gru2_merged = layers.concatenate([gru_2, gru_2b])

# Çıkış katmanı
inner = layers.Dense(num_chars + 1, kernel_initializer='he_normal', activation='softmax')(gru2_merged)

# CTC kaybı fonksiyonu
def ctc_lambda_func(args):
    y_pred, label_length, input_length, labels = args
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

# Kayıp katmanı
loss_out = layers.Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([inner, label_length, input_length, the_labels])

# Modelin tanımlanması
model = keras.models.Model(inputs={'input_data': input_data,
                                   'the_labels': the_labels,
                                   'input_length': input_length,
                                   'label_length': label_length},
                           outputs=loss_out)

# 10. Modelin Derlenmesi
model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer='adam')

# 11. Veri Üreticilerinin Oluşturulması
train_generator = DataGenerator(train_images, train_labels, train_label_sequences, batch_size, img_width, img_height, max_label_length)
val_generator = DataGenerator(val_images, val_labels, val_label_sequences, batch_size, img_width, img_height, max_label_length)

# 12. Modelin Eğitilmesi
model.fit(train_generator,
          validation_data=val_generator,
          epochs=10)

# 13. Tahmin Modelinin Oluşturulması
prediction_model = keras.models.Model(inputs=input_data, outputs=inner)

# 14. Tahminlerin Çözümlenmesi
def decode_batch_predictions(pred):
    input_len = np.ones(pred.shape[0]) * pred.shape[1]
    results = K.ctc_decode(pred, input_length=input_len, greedy=True)[0][0]
    output_text = []

    for res in results:
        res = res.numpy()
        decoded = ''.join([num_to_char[int(r)] for r in res if int(r) != -1])
        output_text.append(decoded)

    return output_text

# 15. Modelin Test Edilmesi
for batch in val_generator:
    batch_images = batch[0]['input_data']
    preds = prediction_model.predict(batch_images)
    pred_texts = decode_batch_predictions(preds)

    for i in range(len(pred_texts)):
        print("Gerçek Etiket:", val_labels[i])
        print("Tahmin Edilen Etiket:", pred_texts[i])
    break


Benzersiz karakter sayısı: 34
Karakterler: ['-', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'V', 'Y', 'Z']
Maksimum etiket uzunluğu: 9
Eğitim örnek sayısı: 90000
Doğrulama örnek sayısı: 10000
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1407/1407 ━━━━━━━━━━━━━━━━━━━━ 2130s 2s/step - loss: 17.2874 - val_loss: 1.7435
Epoch 2/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 2136s 2s/step - loss: 1.2124 - val_loss: 0.4735
Epoch 3/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 2175s 2s/step - loss: 0.3562 - val_loss: 0.2534
Epoch 4/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 2166s 1s/step - loss: 0.1597 - val_loss: 0.1332
Epoch 5/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 2152s 2s/step - loss: 0.0986 - val_loss: 0.0986
Epoch 6/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 2158s 2s/step - loss: 0.0787 - val_loss: 0.1475
Epoch 7/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 2109s 1s/step - loss: 0.0610 - val_loss: 0.0728
Epoch 8/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 2092s 1s/step - loss: 0.0629 - val_loss: 0.0852
Epoch 9/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 2115s 1s/step - loss: 0.0574 - val_loss: 0.0843
Epoch 10/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 2100s 1s/step - loss: 0.0468 - val_loss: 0.0634
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 130ms/step
Gerçek Etiket: 77-G-7115
Tahmin Edilen Etiket: 77-G-7115
Gerçek Etiket

In [ ]:
# Tahmin modelini kaydetme
prediction_model.save('plaka_tanima_modeli.h5')


In [ ]:
# Tahmin Modelini Oluşturma
prediction_model = keras.models.Model(inputs=input_data, outputs=inner)


In [ ]:
# Tahmin modelini kaydetme
prediction_model.save('license_plate_recognition_model.h5')


In [ ]:
import pickle

# Karakter-Numara Haritalarını Kaydetme
with open('char_to_num.pkl', 'wb') as f:
    pickle.dump(char_to_num, f)

with open('num_to_char.pkl', 'wb') as f:
    pickle.dump(num_to_char, f)
